## News :D



In [2]:
%load_ext autoreload
%autoreload 2

import requests
import re 
import itertools
from bs4 import BeautifulSoup # Imports bs4
import wget # Imports wget
import articles
import sitemaps
import test_functions

# Data

### Data formats

CNN has links by month

NYT has links by day

BI has links by month

TH has links by day

NYP has links by day

DM has links by day

Fox has links by page num

### Thoughts on how to organize the data 

we use a pandas dataframe organized like this

| Dates | CNN | NYT | BI | TH | NYP | DM | FOX |
| ----------- | ------------------------------------------ | ------- | ------- | ------- | ------- | ------- | ------- |
| 2014-01-01  | list of tuples with article link & title   | ... | ... | ... | ... | ... | ... |
| 2014-01-02  | ... | ... | ... | ... | ... | ... | ... |
| 2014-01-03  | ... | ... | ... | ... | ... | ... | ... |
| 2014-01-04  | ... | ... | ... | ... | ... | ... | ... |
| ...         | ... | ... | ... | ... | ... | ... | ... |
| 2024-12-31  | ... | ... | ... | ... | ... | ... | ... |

### Code to get data
CNN:
```html
<li>
    <span class="date">2024-01-31</span>
    <span class="sitemap-link">
        <a href="https://www.cnn.com/tech/live-news/meta-x-discord-tiktok-snap-chiefs-testimony-senate/index.html">
            Mark Zuckerberg apologizes to families over social media harms in contentious Senate hearing
        </a>
    </span>
</li>
```

NYT:
```html
<li>
    <a href="https://www.nytimes.com/2023/12/31/business/dealbook/pga-tour-saudi-deal-deadline.html">
        PGA Tour and Saudi-Backed LIV Extend Deadline to Finalize Deal
    </a>
</li>
```

BI:
```html
<p>
    <a href="https://www.businessinsider.com/feud-capote-vs-swans-ann-woodward-suicide-death-real-story-2024-1">
        The real story behind the suicide of American socialite Ann Woodward in &#39;Feud: Capote vs. The Swans&#39;
    </a>
    <br>2024-02-01T03:31:01.68Z
</p>
```

NYP:
```html
<h3 class="story__headline headline headline--archive">
    <a href="https://nypost.com/2024/01/01/sports/og-anunoby-trending-toward-the-perfect-piece-for-knicks/" rel="" target="_self">
		Knicks&#039; big trade acquisition already looks like the &#039;perfect fit&#039; 												
    </a>
</h3>
```

DM:
```html
<a href="/wires/ap/article-12917573/Christian-McCaffrey-miss-49ers-regular-season-finale-return-playoffs.html" title="">
    Christian McCaffrey will miss the 49ers regular-season finale but should return for playoffs
</a>
```

FOX:
```html
<loc>
    https://www.foxnews.com/opinion/why-the-department-of-education-is-going-to-rip
</loc>
<lastmod>
    2025-03-12T05:00:54-04:00
</lastmod>
```

In [3]:
[cnn_dict, nyt_dict, bi_dict, nyp_dict, dm_dict, fox_links] = sitemaps.sitemaps()

In [48]:
start_year = 2020
end_year = 2023
start_month = 1
end_month = 12
news = [1,1,1,0,0,0] # determines which news sources we scrape (for testing)       0 = don't   1 = do       CNN, NYT, BI, NYP, DM, FOX

[cnn_articles, nyt_articles, bi_articles, nyp_articles, dm_articles, fox_articles] = articles.articles(news,start_year,end_year,start_month,end_month)

KeyboardInterrupt: 

In [43]:
# Pandas Testing area
import pandas as pd
import numpy as np

mday = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

news_names = ["CNN", "NYT", "BI", "NYP", "DM", "FOX"]
news_data = {}
example_data = {}

for year in range(start_year,end_year+1):
    for month in range(start_month,end_month+1):
        for day in range(mday[month-1]):
            news_data[f"{year}-{str(month).zfill(2)}-{str(day+1).zfill(2)}"] = [
                cnn_articles[f"{year}-{str(month).zfill(2)}-{str(day+1).zfill(2)}"],
                nyt_articles[f"{year}-{str(month).zfill(2)}-{str(day+1).zfill(2)}"],
                bi_articles[f"{year}-{str(month).zfill(2)}-{str(day+1).zfill(2)}"],
                'N/A',
                'N/A',
                'N/A'
            ]
            example_data[f"{year}-{str(month).zfill(2)}-{str(day+1).zfill(2)}"] = [
                'list',
                'list',
                'list',
                'list',
                'list',
                'list'
            ]

test_frame = pd.DataFrame(news_data, index=news_names)
example_test_frame = pd.DataFrame(example_data, index=news_names)

In [22]:
test_news = [1,1,0,0,0,0]

[words, title_counts] = test_functions.most_common(test_news,start_year,end_year,start_month,end_month,'Trump',test_frame)

for source in words:
    del_words = []

    for i in words[source]:
        words[source][i] = (words[source][i] / title_counts[source]) * 100

    for i in words[source]:
        if words[source][i] <= 1:
            del_words.append(i)
    for i in del_words:
        del words[source][i]


dels = []
for i in words['CNN']:
    if i not in words['NYT']:
        dels.append(i)
for i in dels:
    del words['CNN'][i]

dels = []
for i in words['NYT']:
    if i not in words['CNN']:
        dels.append(i)
for i in dels:
    del words['NYT'][i]

for word in words['CNN']:
    print(word)
    print(f"CNN: {words['CNN'][word]}")
    print(f"NYT: {words['NYT'][word]}")
    print("")


Bloomberg
CNN: 1.1764705882352942
NYT: 1.675977653631285

Biden
CNN: 1.1764705882352942
NYT: 1.9553072625698324

John
CNN: 1.411764705882353
NYT: 1.675977653631285

Davos
CNN: 1.647058823529412
NYT: 1.9553072625698324

but
CNN: 1.647058823529412
NYT: 1.675977653631285

China
CNN: 1.647058823529412
NYT: 2.793296089385475

Parnas
CNN: 1.647058823529412
NYT: 1.1173184357541899

Why
CNN: 1.8823529411764703
NYT: 1.9553072625698324

New
CNN: 2.1176470588235294
NYT: 2.2346368715083798

Middle
CNN: 2.3529411764705883
NYT: 1.675977653631285

East
CNN: 2.3529411764705883
NYT: 1.675977653631285

What
CNN: 2.588235294117647
NYT: 4.748603351955307

Iranian
CNN: 2.823529411764706
NYT: 1.3966480446927374

by
CNN: 3.058823529411765
NYT: 2.5139664804469275

Impeachment
CNN: 3.058823529411765
NYT: 13.128491620111731

Ukraine
CNN: 3.5294117647058822
NYT: 3.35195530726257

How
CNN: 4.0
NYT: 2.793296089385475

Bolton
CNN: 4.0
NYT: 2.793296089385475

as
CNN: 4.235294117647059
NYT: 5.307262569832402

US
CNN:

In [ ]:
test_news = [1,1,1,0,0,0]

[values, title_counts] = test_functions.sentiment(test_news,start_year,end_year,start_month,end_month,'Trump',test_frame)

print("")
print(f"CNN Positive: {values['CNN']['pos']}")
print(f"NYT Positive: {values['NYT']['pos']}")
print(f"BI Positive: {values['BI']['pos']}")

print("")
print(f"CNN Negative: {values['CNN']['neg']}")
print(f"NYT Negative: {values['NYT']['neg']}")
print(f"BI Negative: {values['BI']['neg']}")

print("")
print(f"CNN Neutral: {values['CNN']['neu']}")
print(f"NYT Neutral: {values['NYT']['neu']}")
print(f"BI Neutral: {values['BI']['neu']}")

print("")
print(f"CNN Compound: {values['CNN']['compound']}")
print(f"NYT Compound: {values['NYT']['compound']}")
print(f"BI Compound: {values['BI']['compound']}")


CNN Positive: 0.09536363636363637
NYT Positive: 0.0906842105263158
BI Positive: 0.040777777777777774

CNN Negative: 0.10754545454545453
NYT Negative: 0.1446842105263158
BI Negative: 0.067

CNN Neutral: 0.7970909090909092
NYT Neutral: 0.7646842105263157
BI Neutral: 0.8923333333333334

CNN Compound: 0.03554545454545455
NYT Compound: -0.1005736842105263
BI Compound: -0.15686666666666665
